# Mecanisme de atenție și transformatoare

Un dezavantaj major al rețelelor recurente este că toate cuvintele dintr-o secvență au același impact asupra rezultatului. Acest lucru duce la o performanță sub-optimală în cazul modelelor standard LSTM encoder-decoder pentru sarcini de tip secvență-la-secvență, cum ar fi Recunoașterea Entităților Numite și Traducerea Automată. În realitate, anumite cuvinte din secvența de intrare au adesea un impact mai mare asupra ieșirilor secvențiale decât altele.

Să luăm în considerare un model secvență-la-secvență, cum ar fi traducerea automată. Acesta este implementat prin două rețele recurente, unde o rețea (**encoder**) comprimă secvența de intrare într-o stare ascunsă, iar cealaltă, **decoder**, desfășoară această stare ascunsă pentru a genera rezultatul tradus. Problema cu această abordare este că starea finală a rețelei întâmpină dificultăți în a-și aminti începutul propoziției, ceea ce duce la o calitate slabă a modelului pentru propoziții lungi.

**Mecanismele de atenție** oferă o modalitate de a pondera impactul contextual al fiecărui vector de intrare asupra fiecărei predicții de ieșire a RNN-ului. Acest lucru este implementat prin crearea unor scurtături între stările intermediare ale RNN-ului de intrare și RNN-ului de ieșire. Astfel, atunci când generăm simbolul de ieșire $y_t$, vom lua în considerare toate stările ascunse de intrare $h_i$, cu coeficienți de greutate diferiți $\alpha_{t,i}$. 

![Imagine care arată un model encoder/decoder cu un strat de atenție aditiv](../../../../../translated_images/ro/encoder-decoder-attention.7a726296894fb567.webp)
*Modelul encoder-decoder cu mecanism de atenție aditiv din [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citat din [acest articol de blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matricea de atenție $\{\alpha_{i,j}\}$ reprezintă gradul în care anumite cuvinte de intrare contribuie la generarea unui cuvânt dat în secvența de ieșire. Mai jos este un exemplu al unei astfel de matrici:

![Imagine care arată o aliniere exemplară găsită de RNNsearch-50, preluată din Bahdanau - arviz.org](../../../../../translated_images/ro/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figura preluată din [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Mecanismele de atenție sunt responsabile pentru o mare parte din starea actuală sau aproape actuală a artei în procesarea limbajului natural. Totuși, adăugarea atenției crește considerabil numărul de parametri ai modelului, ceea ce a dus la probleme de scalare cu RNN-urile. O constrângere cheie a scalării RNN-urilor este că natura recurentă a modelelor face dificilă gruparea și paralelizarea antrenamentului. Într-un RNN, fiecare element al unei secvențe trebuie procesat în ordine secvențială, ceea ce înseamnă că nu poate fi paralelizat cu ușurință.

Adoptarea mecanismelor de atenție, combinată cu această constrângere, a dus la crearea modelelor Transformatoare, acum considerate Starea de Artă, pe care le cunoaștem și le folosim astăzi, de la BERT la OpenGPT3.

## Modele Transformatoare

În loc să transmită contextul fiecărei predicții anterioare în pasul următor de evaluare, **modelele transformatoare** folosesc **codificări poziționale** și **atenție** pentru a captura contextul unei intrări date într-o fereastră de text furnizată. Imaginea de mai jos arată cum codificările poziționale împreună cu atenția pot captura contextul într-o fereastră dată.

![GIF animat care arată cum sunt realizate evaluările în modelele transformatoare.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Deoarece fiecare poziție de intrare este mapată independent la fiecare poziție de ieșire, transformatoarele pot paraleliza mai bine decât RNN-urile, ceea ce permite modele de limbaj mult mai mari și mai expresive. Fiecare cap de atenție poate fi utilizat pentru a învăța relații diferite între cuvinte, ceea ce îmbunătățește sarcinile de procesare a limbajului natural.

## Construirea unui Model Transformator Simplu

Keras nu conține un strat Transformator încorporat, dar putem construi unul propriu. Ca și înainte, ne vom concentra pe clasificarea textului din setul de date AG News, dar merită menționat că modelele Transformatoare oferă cele mai bune rezultate pentru sarcini NLP mai dificile.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Noile straturi în Keras ar trebui să fie subclase ale clasei `Layer` și să implementeze metoda `call`. Să începem cu stratul **Positional Embedding**. Vom folosi [un cod din documentația oficială Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Vom presupune că umplem toate secvențele de intrare la lungimea `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Acest strat constă din două straturi `Embedding`: unul pentru încorporarea token-urilor (în modul pe care l-am discutat anterior) și unul pentru pozițiile token-urilor. Pozițiile token-urilor sunt create ca o secvență de numere naturale de la 0 la `maxlen` folosind `tf.range`, apoi sunt trecute prin stratul de încorporare. Cele două vectori de încorporare rezultați sunt apoi adunați, producând o reprezentare încorporată pozițional a intrării cu forma `maxlen`$\times$`embed_dim`.

Acum, să implementăm blocul transformer. Acesta va prelua ieșirea stratului de încorporare definit anterior:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer aplică `MultiHeadAttention` pe intrarea codificată pozițional pentru a produce vectorul de atenție cu dimensiunea `maxlen`$\times$`embed_dim`, care este apoi combinat cu intrarea și normalizat folosind `LayerNormalization`.

> **Notă**: `LayerNormalization` este similar cu `BatchNormalization` discutat în partea *Computer Vision* a acestui parcurs de învățare, dar normalizează ieșirile stratului anterior pentru fiecare eșantion de antrenament în mod independent, pentru a le aduce în intervalul [-1..1].

Ieșirea acestui strat este apoi trecută printr-o rețea `Dense` (în cazul nostru - un perceptron cu două straturi), iar rezultatul este adăugat la ieșirea finală (care este din nou supusă normalizării).


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Modele Transformer BERT

**BERT** (Bidirectional Encoder Representations from Transformers) este o rețea de transformatoare foarte mare, cu mai multe straturi, având 12 straturi pentru *BERT-base* și 24 pentru *BERT-large*. Modelul este mai întâi pre-antrenat pe un corpus mare de date textuale (Wikipedia + cărți) folosind antrenare nesupravegheată (prezicerea cuvintelor mascate într-o propoziție). În timpul pre-antrenării, modelul dobândește un nivel semnificativ de înțelegere a limbajului, care poate fi apoi valorificat cu alte seturi de date prin ajustare fină. Acest proces se numește **învățare transferabilă**.

![imagine de pe http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ro/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Există multe variații ale arhitecturilor Transformer, inclusiv BERT, DistilBERT, BigBird, OpenGPT3 și altele, care pot fi ajustate fin.

Să vedem cum putem folosi un model BERT pre-antrenat pentru a rezolva problema noastră tradițională de clasificare a secvențelor. Vom împrumuta ideea și ceva cod din [documentația oficială](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Pentru a încărca modele pre-antrenate, vom folosi **Tensorflow hub**. Mai întâi, să încărcăm vectorizatorul specific BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Este important să folosești același vectorizator ca cel pe care a fost antrenată rețeaua originală. De asemenea, vectorizatorul BERT returnează trei componente:
* `input_word_ids`, care este o secvență de numere de tokeni pentru propoziția de intrare
* `input_mask`, care indică ce parte a secvenței conține intrarea reală și care parte este umplere. Este similar cu masca produsă de stratul `Masking`
* `input_type_ids` este utilizat pentru sarcini de modelare a limbajului și permite specificarea a două propoziții de intrare într-o singură secvență.

Apoi, putem instanția extractorul de caracteristici BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Așadar, stratul BERT returnează o serie de rezultate utile:
* `pooled_output` este rezultatul mediei tuturor token-urilor din secvență. Îl poți considera ca o reprezentare semantică inteligentă a întregii rețele. Este echivalent cu ieșirea stratului `GlobalAveragePooling1D` din modelul nostru anterior.
* `sequence_output` este ieșirea ultimului strat transformer (corespunde ieșirii `TransformerBlock` din modelul nostru de mai sus).
* `encoder_outputs` sunt ieșirile tuturor straturilor transformer. Deoarece am încărcat un model BERT cu 4 straturi (așa cum probabil îți dai seama din nume, care conține `4_H`), acesta are 4 tensori. Ultimul este același cu `sequence_output`.

Acum vom defini modelul de clasificare end-to-end. Vom folosi *definirea funcțională a modelului*, în care definim intrarea modelului și apoi oferim o serie de expresii pentru a calcula ieșirea acestuia. De asemenea, vom face ca greutățile modelului BERT să nu fie antrenabile și vom antrena doar clasificatorul final:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Deși există puțini parametri antrenabili, procesul este destul de lent, deoarece extractorul de caracteristici BERT este foarte solicitant din punct de vedere computațional. Se pare că nu am reușit să obținem o acuratețe rezonabilă, fie din cauza lipsei de antrenament, fie din cauza lipsei de parametri ai modelului.

Să încercăm să deblocăm greutățile BERT și să le antrenăm și pe acestea. Acest lucru necesită o rată de învățare foarte mică și, de asemenea, o strategie de antrenament mai atentă, cu **warmup**, folosind optimizatorul **AdamW**. Vom folosi pachetul `tf-models-official` pentru a crea optimizatorul:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Așa cum se poate observa, antrenamentul merge destul de încet - dar s-ar putea să vrei să experimentezi și să antrenezi modelul pentru câteva epoci (5-10) și să vezi dacă poți obține cel mai bun rezultat comparativ cu abordările pe care le-am folosit anterior.

## Biblioteca Huggingface Transformers

O altă modalitate foarte comună (și puțin mai simplă) de a utiliza modelele Transformer este [pachetul HuggingFace](https://github.com/huggingface/), care oferă blocuri simple pentru diferite sarcini NLP. Este disponibil atât pentru Tensorflow, cât și pentru PyTorch, un alt cadru foarte popular pentru rețele neuronale.

> **Notă**: Dacă nu ești interesat să vezi cum funcționează biblioteca Transformers - poți sări direct la finalul acestui notebook, deoarece nu vei vedea nimic substanțial diferit față de ceea ce am făcut mai sus. Vom repeta aceiași pași de antrenare a modelului BERT folosind o bibliotecă diferită și un model substanțial mai mare. Prin urmare, procesul implică un antrenament destul de lung, așa că s-ar putea să vrei doar să te uiți peste cod.

Să vedem cum poate fi rezolvată problema noastră folosind [Huggingface Transformers](http://huggingface.co).


Primul lucru pe care trebuie să-l facem este să alegem modelul pe care îl vom folosi. Pe lângă câteva modele integrate, Huggingface conține un [repository online de modele](https://huggingface.co/models), unde poți găsi multe alte modele pre-antrenate de comunitate. Toate aceste modele pot fi încărcate și utilizate doar prin furnizarea unui nume de model. Toate fișierele binare necesare pentru model vor fi descărcate automat.

În anumite situații, va trebui să încarci propriile tale modele, caz în care poți specifica directorul care conține toate fișierele relevante, inclusiv parametrii pentru tokenizer, fișierul `config.json` cu parametrii modelului, greutățile binare etc.

Pornind de la numele modelului, putem inițializa atât modelul, cât și tokenizer-ul. Să începem cu un tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Obiectul `tokenizer` conține funcția `encode` care poate fi utilizată direct pentru a codifica textul:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Putem folosi, de asemenea, tokenizer-ul pentru a codifica o secvență într-un mod potrivit pentru a fi transmis modelului, adică incluzând câmpurile `token_ids`, `input_mask`, etc. De asemenea, putem specifica că dorim tensori Tensorflow prin furnizarea argumentului `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

În cazul nostru, vom folosi un model BERT pre-antrenat numit `bert-base-uncased`. *Uncased* indică faptul că modelul este insensibil la majuscule și minuscule.

Când antrenăm modelul, trebuie să furnizăm o secvență tokenizată ca intrare, așa că vom proiecta un flux de procesare a datelor. Deoarece `tokenizer.encode` este o funcție Python, vom folosi aceeași abordare ca în unitatea anterioară, apelând-o cu ajutorul `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Acum putem încărca modelul actual folosind pachetul `BertForSequenceClassification`. Acest lucru asigură că modelul nostru are deja o arhitectură necesară pentru clasificare, inclusiv clasificatorul final. Veți vedea un mesaj de avertizare care indică faptul că greutățile clasificatorului final nu sunt inițializate, iar modelul ar necesita pre-antrenare - acest lucru este perfect în regulă, deoarece exact asta urmează să facem!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Așa cum puteți vedea din `summary()`, modelul conține aproape 110 milioane de parametri! Probabil, dacă dorim o sarcină simplă de clasificare pe un set de date relativ mic, nu dorim să antrenăm stratul de bază BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Acum suntem gata să începem antrenamentul!

> **Notă**: Antrenarea unui model BERT la scară completă poate fi foarte consumatoare de timp! Prin urmare, vom antrena modelul doar pentru primele 32 de loturi. Acesta este doar un exemplu pentru a arăta cum se configurează antrenamentul modelului. Dacă ești interesat să încerci antrenamentul la scară completă - elimină pur și simplu parametrii `steps_per_epoch` și `validation_steps`, și pregătește-te să aștepți!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Dacă mărești numărul de iterații, aștepți suficient și antrenezi pentru mai multe epoci, te poți aștepta ca clasificarea cu BERT să ne ofere cea mai bună acuratețe! Asta deoarece BERT înțelege deja destul de bine structura limbajului, iar noi trebuie doar să ajustăm clasificatorul final. Totuși, deoarece BERT este un model mare, întregul proces de antrenare durează mult și necesită o putere de calcul considerabilă! (GPU, și de preferat mai multe).

> **Note:** În exemplul nostru, am folosit unul dintre cele mai mici modele BERT pre-antrenate. Există modele mai mari care probabil vor oferi rezultate mai bune.


## Concluzii

În această unitate, am analizat arhitecturi de modele foarte recente bazate pe **transformers**. Le-am aplicat pentru sarcina noastră de clasificare a textului, dar, în mod similar, modelele BERT pot fi utilizate pentru extragerea entităților, răspunsuri la întrebări și alte sarcini de procesare a limbajului natural.

Modelele transformer reprezintă vârful de performanță actual în NLP, iar în majoritatea cazurilor ar trebui să fie prima soluție pe care începeți să o experimentați atunci când implementați soluții personalizate de NLP. Totuși, înțelegerea principiilor de bază ale rețelelor neuronale recurente discutate în acest modul este extrem de importantă dacă doriți să construiți modele neuronale avansate.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
